# Python based analysis

This script takes trees as inputs and writes another sets of trees as outputs.<br>
The trees are filtered in between.

In [1]:
import os, sys
import numpy as np
import uproot
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
#Global parameters

basedir = '../ROOT_FILES/treesWithNN/'
injob  = 'baseline/tree_2018_UL_baseline'
outjob = 'sr/tree_2018_UL_sr'
campaign = '2018_UL'
nnscore = 'nnscore_qcd_vlld_2018'

indir = os.path.join(basedir, injob)
outdir = os.path.join(basedir, outjob)
os.makedirs(outdir, exist_ok=True) 

In [3]:
import uproot
import pandas as pd
from tqdm.notebook import tqdm

def read_file_into_df(filepath, step_size=100000):
    with uproot.open(filepath) as tfile:
        if "myEvents" not in tfile:
            print(f"Warning: 'myEvents' tree not found in {filepath}. Skipping.")
            return pd.DataFrame()

        ttree = tfile["myEvents"]
        total_entries = ttree.num_entries

        if total_entries == 0:
            print(f"Warning: 'myEvents' tree in {filepath} is empty. Skipping.")
            return pd.DataFrame()

        total_batches = (total_entries + step_size - 1) // step_size
        dfs = []

        with tqdm(total=total_batches, unit=" batch", desc="Reading", colour='green', dynamic_ncols=False, mininterval=0.5) as pbar:
            for batch in ttree.iterate(ttree.keys(), step_size=step_size, library="pd"):
                dfs.append(batch)
                pbar.update(1)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

def write_df_into_file(df, filepath, step_size=100000):
    if df.empty:
        print(f"Warning: Attempting to write empty DataFrame to {filepath}. Skipping.")
        return

    total_entries = len(df)
    total_batches = (total_entries + step_size - 1) // step_size

    with tqdm(total=total_batches, unit=" batch", desc="Reading", colour='green', dynamic_ncols=False, mininterval=0.5) as pbar:
        with uproot.recreate(filepath) as file:
            for start in range(0, total_entries, step_size):
                chunk = df.iloc[start:start + step_size]
                if start == 0: file["myEvents"] = {col: chunk[col].to_numpy() for col in df.columns}
                else:          file["myEvents"].extend({col: chunk[col].to_numpy() for col in df.columns})
                pbar.update(1)

print('Functions loaded.')

Functions loaded.


In [4]:
%%time

list_of_files = os.listdir(indir)
list_empty_input = []
list_empty_output = []

for f in list_of_files:

    #if not 'QCD' in f: continue
    if not f.endswith('.root'): continue
    #if 'SingleMuon' not in f: continue

    #Step1: Prepare the dataframe
    print(f'\nProcessing file: {f}')
    filepath = os.path.join(indir, f)
    sample = f.split("_")[1]
    subsample = "_".join(f.split("_")[2:])
       
    outfile = os.path.join(outdir, f)
    #if os.path.exists(outfile): continue
    #if 'WJets' not in f: continue

    df = read_file_into_df(filepath)
    if df.empty : 
        #print(f"\033[0;31mWarning: Empty input file: {filepath}\033[0m\n")
        list_empty_input.append(filepath)
        continue

    ####################################
    #         EVENT SELECTION          #
    ####################################

    #Step1: Controlling QCD:
    dy_veto = 'not (channel == 3 and 76 < dilep_mass < 106)'
    qcd_cr = f'{dy_veto} and {nnscore}<0.30 and 0.02<lep0_iso<0.15 and lep0_sip3d>5'
    qcd_vr = f'{dy_veto} and {nnscore}<0.30 and 0.02<lep0_iso<0.15 and lep0_sip3d<5'

    #Step2: Controlling Drell-Yan:
    dycr  = f'76<dilep_mass<106  and dilep_ptratio > 0.7'

    #Step3: Filter bad events:
    tight_sip3d =  'lep0_sip3d<5 and lep1_sip3d<10'
    good_events = f'{tight_sip3d} and {dy_veto} and {nnscore}>0.30 and HT>50' 
    
    #Step4: Controlling TTbar:
    top_cr = f'{good_events} and {nnscore}>0.70 and nbjet>0'    

    #Step5: Validation:
    val_region = f'{good_events} and 0.50<{nnscore}<0.70'

    #Step6: Signal regions:
    sr = f'{good_events} and {nnscore}>0.70 and nbjet==0'
    
    #------------------------------
    # Final event selection:
    event_selection = sr
    #------------------------------

    # Use df.query with the combined condition
    df_filtered = df.query(event_selection)
    
    if df_filtered.empty: list_empty_output.append(f'{sample}_{subsample}')

    nbefore = len(df)
    nafter = len(df_filtered)

    frac = 0
    if nbefore != 0: frac = nafter*100/nbefore

    #display(df_filtered)

    write_df_into_file(df_filtered, outfile)
    print(f'\033[1;32mFile written: {outfile}\033[0m ({nbefore} -> {nafter}, {frac:.2f}%)\n')
    #break #file

print(f'\n\033[1;33mSummary:\033[0m')
print(f'\n\033[33mEmpty input: {list_empty_input}\033[0m')
print(f'\n\033[31mEmpty output: {list_empty_output}\033[0m\n')


Processing file: tree_DYJetsToLL_M10to50.root


Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

File written: ../ROOT_FILES/treesWithNN/sr/tree_2018_UL_sr/tree_DYJetsToLL_M10to50.root (257 -> 14, 5.45%)


Processing file: tree_DYJetsToLL_M50.root


Reading:   0%|          | 0/2 [00:00<?, ? batch/s]

...

Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

File written: ../ROOT_FILES/treesWithNN/sr/tree_2018_UL_sr/tree_ZZ_ZZTo2Q2Nu.root (28 -> 1, 3.57%)


Processing file: tree_ZZ_ZZTo4L.root


Reading:   0%|          | 0/15 [00:00<?, ? batch/s]

Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

File written: ../ROOT_FILES/treesWithNN/sr/tree_2018_UL_sr/tree_ZZ_ZZTo4L.root (1471725 -> 75740, 5.15%)


Summary:

Empty input: []

Empty output: ['QCD_EMEnriched_170to300.root', 'QCD_EMEnriched_300toInf.root', 'QCD_EMEnriched_50to80.root', 'QCD_EMEnriched_80to120.root', 'QCD_MuEnriched_20to30.root', 'QCD_MuEnriched_30to50.root', 'QCD_MuEnriched_50to80.root', 'WW_WWTo4Q.root']

CPU times: user 26.5 s, sys: 2.76 s, total: 29.2 s
Wall time: 48.6 s


In [5]:
%%time

run_here = False

channels = ['mm', 'me', 'em', 'ee']
for channel_ in channels:
    jobname_ = outjob
    campaign_ = campaign
    dump_ = outjob.replace('tree', 'hist')+'_'+channel_
    arguments = f'extractHistsFromTrees.C("{jobname_}", "{dump_}", "{campaign_}", "{channel_}")'
    command = f"root -q -b -l '{arguments}'"
    print(f'\033[93m\nRunnning .. \n{command}\033[0m\n')

    if run_here: os.system(command)
    else: print('Run it yourself:')
    
print('Done!')


Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("sr/tree_2018_UL_sr", "sr/hist_2018_UL_sr_mm", "2018_UL", "mm")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("sr/tree_2018_UL_sr", "sr/hist_2018_UL_sr_me", "2018_UL", "me")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("sr/tree_2018_UL_sr", "sr/hist_2018_UL_sr_em", "2018_UL", "em")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("sr/tree_2018_UL_sr", "sr/hist_2018_UL_sr_ee", "2018_UL", "ee")'

Run it yourself:
Done!
CPU times: user 190 μs, sys: 0 ns, total: 190 μs
Wall time: 178 μs
